In [77]:
data <- read.csv("test.csv")

In [78]:
#必要欄位：行銷活動名稱、廣告名稱、觸及人數、支出金額、不重複點擊次數、成果
head(data)

,分析報告開始,分析報告結束,廣告名稱,投放狀態,成果,成果類型,觸及人數,頻率,每次成果的成本..USD.,支出金額..USD.,⋯,CTR.全部.,CPC.全部...USD.,瀏覽次數,CPM.每千次廣告曝光成本...USD.,連結點擊次數,CPC.連結...USD.,CTR.連結.,行銷活動名稱,不重複點擊次數.全部.,廣告組合名稱
1,2015-12-30,2016-01-20,v01,active,269,actions:mobile_app_install,61444,1.779816,2.930967,788.43,⋯,0.4782414,1.507514,109359,7.209558,493,1.599249,0.4508088,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,445,v_wusha_A_15-44_cpm
2,2015-12-30,2016-01-20,v02,active,234,actions:mobile_app_install,58665,1.549339,2.768333,647.79,⋯,0.5225982,1.363768,90892,7.12703,427,1.517073,0.4697883,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,427,v_wusha_A_15-44_cpm
3,2015-12-30,2016-01-20,v02,active,0,actions:mobile_app_install,3,1,0,0.01,⋯,0,0,3,3.333333,NA,NA,NA,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,0,v_AC064_A_15-44_cpm
4,2015-12-30,2016-01-20,c05,active,503,actions:mobile_app_install,175026,1.981871,3.285348,1652.53,⋯,0.542264,0.878538,346879,4.763996,1690,0.9778284,0.4872016,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,1729,mix_wusha_A_15-44_cpm
5,2015-12-30,2016-01-20,c03,active,113,actions:mobile_app_install,65203,1.761453,2.326018,262.84,⋯,0.426636,0.5364082,114852,2.28851,408,0.6442157,0.3552398,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,453,mix_wusha_A_15-44_cpm
6,2015-12-30,2016-01-20,c02,active,224,actions:mobile_app_install,91316,1.940098,3.551518,795.54,⋯,0.5373613,0.8356513,177162,4.490466,824,0.9654612,0.465111,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,871,mix_wusha_A_15-44_cpm


In [79]:
#資料合併

library(plyr)

a <- ddply(data, c("行銷活動名稱","廣告名稱"), summarize,  
           people=sum(觸及人數), 
           money=sum(支出金額..USD.), 
           click=sum(不重複點擊次數.全部.), 
           result=sum(成果))

head(a)

,行銷活動名稱,廣告名稱,people,money,click,result
1,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c02,105786,850.17,973,241
2,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c03,186026,855.4,1597,333
3,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c05,599409,4728.44,6100,1614
4,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,v01,61447,788.43,445,269
5,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,v02,58668,647.8,427,234


In [120]:
#index欄位計算
a$CTR <- (a$click/a$people)*100
a$CPC <- (a$money/a$click)
a$CPI <- (a$money/a$result)
a$CPM <- (a$money/a$people)*1000

b <- a
head(b)

,行銷活動名稱,廣告名稱,people,money,click,result,CTR,CPC,CPI,CPM
1,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c02,105786,850.17,973,241,0.9197814,0.8737616,3.527676,8.036697
2,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c03,186026,855.4,1597,333,0.8584821,0.5356293,2.568769,4.598282
3,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c05,599409,4728.44,6100,1614,1.017669,0.7751541,2.929641,7.888504
4,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,v01,61447,788.43,445,269,0.7242013,1.771753,2.930967,12.83106
5,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,v02,58668,647.8,427,234,0.7278244,1.517096,2.768376,11.04179


In [127]:
#ntile(rank),result和CTR越大越好,CPC.CPM和CPI越小越好。
#with weight factor

library(data.table)

#b$result <- ntile(b$result, 40) 
b$CTR <- ntile(b$CTR, 10)
b$CPC <- ntile(b$CPC, 5)
b$CPC <- abs(6-b$CPC)
b$CPI <- ntile(b$CPI, 20)
b$CPI <- abs(21-b$CPI)
b$CPM <- ntile(b$CPM, 5)
b$CPM <- abs(6-b$CPM)
b$score <- b$CTR + b$CPC + b$CPI + b$CPM
head(b)

,行銷活動名稱,廣告名稱,people,money,click,result,CTR,CPC,CPI,CPM,score
1,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c02,105786,850.17,973,241,7,3,4,3,17
2,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c03,186026,855.4,1597,333,5,5,20,5,35
3,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,c05,599409,4728.44,6100,1614,9,4,12,4,29
4,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,v01,61447,788.43,445,269,1,1,8,1,11
5,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,v02,58668,647.8,427,234,3,2,16,2,23


In [128]:
c <- b[c("廣告名稱","score")]
c <- setDT(c)[order(-score), Rank := .GRP, -score]
c

,廣告名稱,score,Rank
1,c02,17,4
2,c03,35,1
3,c05,29,2
4,v01,11,5
5,v02,23,3


In [129]:
d <- merge(a, c, by = "廣告名稱")
d <- setDT(d)[order(Rank)]

d

,廣告名稱,行銷活動名稱,people,money,click,result,CTR,CPC,CPI,CPM,score,Rank
1,c03,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,186026,855.4,1597,333,0.8584821,0.5356293,2.568769,4.598282,35,1
2,c05,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,599409,4728.44,6100,1614,1.017669,0.7751541,2.929641,7.888504,29,2
3,v02,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,58668,647.8,427,234,0.7278244,1.517096,2.768376,11.04179,23,3
4,c02,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,105786,850.17,973,241,0.9197814,0.8737616,3.527676,8.036697,17,4
5,v01,遊戲_奇遊_花千骨_IOS_台港澳_01_vic,61447,788.43,445,269,0.7242013,1.771753,2.930967,12.83106,11,5
